# Verdiepende casus
In mijn verdiepende casus ga ik kijken of een behandeling effectief gaat zijn aan de hand van afbeelding van tumoren. <Br>
Mijn data bestaat uit ROI van 512 x 512 met een kleuring of een specifieke cel kleuring zoals CD8+ of PCK. <Br>
Dezelfde ROI heeft meerdere kleuringen, zoals een DAPI kleuring CD8+ en segmentation. <Br>
Deze ga ik alle 3 overlayen om zo de AI te trainen om in het vervolg zelf de CD8 cellen eruit te halen. <br>
IK weet nog niet hoe precies ik dat ga doen. <br>
De DAPI layer wordt gebruikt om nuclei te locaten <Br>
De CD8 layer om te zeggen welke nuclei bij CD8 hoort op basis van intensiteit van het signal <br>
De segmentation of duidelijke te maken waar de celgrenzen zijn en een outline te geven, blauw op zwart <br>
hiervoor ga ik Convolutional Neural Network (CNN) gebruiken <Br> 
test <br>